## Анализ текстов интернет-страниц
### Задача DMP-3643

In [1]:
import datetime
import pandas as pd

In [39]:
#! mv /data1/share/kosm/data/data_all.csv /data1/share/kosm/data/parsed_urls_cred_scor.csv
#df = pd.read_csv('/data1/share/kosm/data/parsed_urls_cred_scor.csv',sep = '\t',skiprows=2,index_col=0)
#for l in open('/data1/share/kosm/data/parsed_urls_cred_scor.csv','r'):
first_row = open('/data1/share/kosm/data/parsed_urls_cred_scor.csv','r').readline()
df = pd.DataFrame([e.split('\t')[1:] for e in open('/data1/share/kosm/data/parsed_urls_cred_scor.csv','r') if not e == first_row])
df.columns ='url init_url response_code title body'.split()

In [82]:
#df['title']

0          Кто подписывает справку 2 НДФЛ - для банка, ди...
1          Пэн: Путешествие в Нетландию / Pan / 2015 / ДБ...
2          Белоснежка и охотник (2012) смотреть онлайн HD...
3                                     Гадание на картах таро
4                                             DNSLookupError
5                          Добро пожаловать в Одноклассники!
6          Купить смартфон Nokia XL Dual Sim RM-1030 в Мо...
7                             ALIEXPRESS SHOPPER | ВКонтакте
8          Рыба запеченная, 416 рецептов + фото рецепты /...
9                              Ляйсан Драчевская | ВКонтакте
10         ИГИЛ показало видео казни «шпионов» четырехлет...
11                                                     embed
12                                             404 Not Found
13                              Сплетница района | ВКонтакте
14         ТЭФИ 2016: Летучая, Сябитова, Меньшова и други...
15                                         Стена | ВКонтакте
16         Кастинги на п

### Предобработка

In [192]:
import re
from pymystem3 import Mystem
from nltk.stem.snowball import SnowballStemmer
import nltk

def preprocess(s):
    return re.sub(u'[^а-яa-z]+',' ',s.lower())

def stem_ru(s, stemmer):
    ''' Simple string stemming.
        Input string is splitted by space and list of basic form of word + part of speech.
        Stemmer is pymystem3.Mystem instance.
    '''
    stemmed_words = [((
                e['analysis'][0]['lex'] 
                if 'analysis' in e and len(e['analysis']) > 0 else ''
             ) 
             + '_' + 
             (
                re.match('^([A-Z]+)', e['analysis'][0]['gr']).group(0) 
                 if 'analysis' in e and len(e['analysis']) > 0 else '')
             )
             for e in stemmer.analyze(s) if len(e['text'].strip()) > 0]
    return [w for w in stemmed_words if w != '_']

def stem_eng(s):
    ''' Simple string stemming.
        Input string is splitted by space and list of basic form of word + part of speech.
        Stemmer is nltk.stem.snowball.SnowballStemmer instance.
    '''
    words = [w for w in re.sub(u'[^a-z]+',' ',s.lower()).split(' ') if len(w)>0]
    return [wordbase + '_' + speechpart  for wordbase,speechpart in nltk.pos_tag(words)]

In [210]:
! mkdir '/data1/share/kosm/data/text_analysis'

In [212]:
from itertools import chain
from collections import Counter
import sklearn
import cPickle

stemmer_ru = Mystem()
df['title_stemmed'] = df['title'].apply(lambda s: stem_ru(s, stemmer_ru) + stem_eng(s))

df[[u'init_url','title_stemmed']].to_csv('/data1/share/kosm/data/text_analysis/stemmed_titles.csv')
st_list = list(chain(*stemmed.values.flat))
counts = Counter(st_list)
word_stat = sorted(counts.items(), key = lambda (word,cnt):-cnt)
cPickle.dump(word_stat, open('/data1/share/kosm/data/text_analysis/stemmed_titles.pck'))

KeyboardInterrupt: 

In [194]:

#df.loc[:20,'title'].apply(lambda s: s.lower())
#re.sub('[^а-яa-z]',' ',s.lower())

text = u'Куплю, новый смартфон * либо поюзанный | nokia   xl dual sim3243 rm- you just selected the prettiest phones with fastest reply rates'

#stem_ru(text, stemmer_ru) + stem_eng(text)
#re.sub(u'[^a-z]+',' ',text.lower()).split(' ')
#print(preprocess(s))

0     [кто_SPRO, подписывать_V, справка_S, для_PR, б...
1     [пэн_S, путешествие_S, в_PR, нетландия_S, кино...
2     [белоснежка_S, и_CONJ, охотник_S, смотреть_V, ...
3                   [гадание_S, на_PR, карта_S, таро_S]
4                                   [dnslookuperror_NN]
5         [добро_S, пожаловать_V, в_PR, одноклассник_S]
6     [купить_V, смартфон_S, в_PR, москва_S, цена_S,...
7              [вконтакте_S, aliexpress_NN, shopper_NN]
8     [рыба_S, запекать_V, рецепт_S, фото_S, рецепт_...
9                 [ляйсан_S, драчевская_S, вконтакте_S]
10    [игил_S, показывать_V, видео_S, казнь_S, шпион...
11                                           [embed_NN]
12                                  [not_RB, found_VBN]
13                  [сплетница_S, район_S, вконтакте_S]
14    [тэфи_S, летучий_A, сябитов_S, меньшов_S, и_CO...
15                               [стена_S, вконтакте_S]
16    [кастинг_S, на_PR, порно_S, категория_S, стран...
17    [парень_S, вдвоем_ADV, трахать_V, по_PR, п

In [188]:
s = text
words = [w for w in re.sub(u'[^a-z]+',' ',s.lower()).split(' ') if len(w)>0]
[wordbase + '_' + speechpart  for wordbase,speechpart in nltk.pos_tag(words)]

[u'nokia_JJ',
 u'xl_NNP',
 u'dual_JJ',
 u'sim_NN',
 u'rm_NN',
 u'you_PRP',
 u'just_RB',
 u'selected_VBD',
 u'the_DT',
 u'pretty_JJ',
 u'phones_NNS',
 u'with_IN',
 u'fastest_JJS',
 u'reply_NN',
 u'rates_NNS']

In [ ]:
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

custom_sent_tokenizer = PunktSentenceTokenizer(text)

tokenized = custom_sent_tokenizer.tokenize(sample_text)

def process_content():
    try:
        for i in tokenized[:5]:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            print(tagged)

    except Exception as e:
        print(str(e))


process_content()

In [178]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
#stemmer = PorterStemmer()

sentence = """At eight o'clock on Thursday morning
    Arthur didn't feel very good."""
tokens = nltk.word_tokenize(re.sub(u'[^a-z]+',' ',text.lower()))
#tokens = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(tokens)
tagged


[(u'nokia', 'JJ'),
 (u'xl', 'NNP'),
 (u'dual', 'JJ'),
 (u'sim', 'NN'),
 (u'rm', 'NN'),
 (u'you', 'PRP'),
 (u'just', 'RB'),
 (u'selected', 'VBD'),
 (u'the', 'DT'),
 (u'pretty', 'JJ'),
 (u'phones', 'NNS'),
 (u'with', 'IN'),
 (u'fastest', 'JJS'),
 (u'reply', 'NN'),
 (u'rates', 'NNS')]

In [153]:
import nltk
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
plurals = ['caresses', 'flies', 'dies', 'mules', 'denied',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
#singles = [stemmer.stem(plural) for plural in plurals]
#print(' '.join(singles))  # doctest: +NORMALIZE_WHITESPACE

print(stemmer.stem(text + ' '.join(plurals)))

куплю, новый смартфон * либо поюзанный | nokia   xl dual sim3243 rm-caresses flies dies mules denied died agreed owned humbled sized meeting stating siezing itemization sensational traditional reference colonizer plot


In [110]:
#import sys, re, pymystem3
#mystem = pymystem3.Mystem()
#mystem.analyze(df.loc[0,'title'])
print('\n'.join('\t'.join(e) for e in stem(df.loc[103,'title'])))


Сливы	слива	S	
маринованные	мариновать	V	
на	на	PR	
зиму	зима	S	


In [145]:
def stem(s,stemmer):
    stemmed_words = [((
                e['analysis'][0]['lex'] 
                if 'analysis' in e and len(e['analysis']) > 0 else ''
             ) 
             + '_' + 
             (
                re.match('^([A-Z]+)', e['analysis'][0]['gr']).group(0) 
                 if 'analysis' in e and len(e['analysis']) > 0 else '')
             )
             for e in stemmer.analyze(s) if len(e['text'].strip()) > 0]
    return [w for w in stemmed_words if w != '_']
print(text)
print('\n'.join(stem(text,m)))

Куплю, новый смартфон * либо поюзанный | nokia   xl dual sim3243 rm-
купить_V
новый_A
смартфон_S
либо_CONJ
поюзать_V


In [ ]:
import json
from pymystem3 import Mystem

>>> text = "Красивая мама красиво мыла раму"
>>> m = Mystem()
>>> lemmas = m.lemmatize(text)

>>> print "lemmas:", ''.join(lemmas)
>>> print "full info:", json.dumps(m.analyze(text), ensure_ascii=False, encoding='utf8')

In [60]:
import sys, re, pymystem3

mystem = pymystem3.Mystem()

def stem(s):
    return [(e['text'].strip(), \
             e['analysis'][0]['lex'] \
                 if 'analysis' in e and len(e['analysis']) > 0 else '', \
             re.match('^([A-Z]+)', e['analysis'][0]['gr']).group(0) \
                 if 'analysis' in e and len(e['analysis']) > 0 else '', \
             ','.join(set(re.findall(r"[\w']+", e['analysis'][0]['gr'])[1:])) \
                 if 'analysis' in e and len(e['analysis']) > 0 else '')\
             for e in mystem.analyze(s) if len(e['text'].strip()) > 0]

line_buf = []
word_buf = []

def stemAndPrint():    
    global word_buf, line_buf
    res = stem(' '.join(word_buf))
    for i, e in enumerate(res):
        if e[0] != word_buf[i]:
            print('Ошибка стемминга', file=sys.stderr)
            print(word_buf, file=sys.stderr)
            print(res, file=sys.stderr)
            sys.exit(1)
        print(line_buf[i] + '\t' + '\t'.join(e[1:]))
    line_buf = []
    word_buf = []
    
c = 0
header = None    
prev_dp_id = None

for line in sys.stdin:
    c += 1
    line = line.strip()
    if c == 1: #header
        header = line.split('\t')
        print(line + '\tlem_word\tpos\tadd_morph')
        continue
    fields = line.split('\t')
    if len(fields) != len(header):
        #TODO exception?
        continue
    r = dict()
    for i, f in enumerate(fields):
        r[header[i]] = f
        
    dp_id = '%s-%s' % (r['dialog_id'], r['phrase_id'])
    if prev_dp_id != dp_id:
        if prev_dp_id is not None:
            stemAndPrint()
        prev_dp_id = dp_id
    line_buf.append(line)
    word_buf.append(r['word'])
    
if prev_dp_id is not None:
    stemAndPrint()

SyntaxError: invalid syntax (<ipython-input-60-c77910493b1d>, line 16)

In [180]:
pi = [1,0,0,0]
q = [3,2,1,0]
eps = 0.05
m = 4
print(eps/4 * sum(q) + (1-eps)*max(q))

2.925


In [182]:
import numpy as np
np.dot(pi,q)

3